In [ ]:
from functions import *

# greeks("NVDA", '2026-01-16', 175, "put")

In [ ]:
continuous_monitor("AMC")

In [ ]:
greeks("AMC", '2026-01-16', 2, "call")

In [ ]:
option_price("AMC", '2026-01-16', 2, "call")

In [ ]:
# Get historical stock price
price = get_stock_price_historical('AAPL', '2024-01-15')
print(f"AAPL price on 2024-01-15: ${price}")

# Calculate theoretical option price on a historical date
option_price = option_price_historical(
    'AAPL', '2024-03-15', 150.0, 'call', '2024-01-15'
)
print(f"Theoretical call price: ${option_price}")

# Calculate Greeks on a historical date
greeks_data = greeks_historical(
    'AAPL', '2024-03-15', 150.0, 'call', '2024-01-15'
)
print(f"Historical Greeks: {greeks_data}")

# With custom implied volatility
option_price = option_price_historical(
    'AAPL', '2024-03-15', 150.0, 'call', '2024-01-15', iv=0.25
)
print(f"Price with 25% IV: ${option_price}")


In [ ]:
# Get intraday stock price
price = get_stock_price_intraday('AAPL', '2025-12-01', '14:30')
print(f"AAPL at 2:30 PM: ${price}")

# Calculate option price at a specific time
option_price = option_price_intraday(
    'AAPL', '2026-01-16', 150.0, 'call', 
    '2025-12-01', '14:30'
)
print(f"Call option price at 2:30 PM: ${option_price}")

# Calculate Greeks at a specific time
greeks = greeks_intraday(
    'AAPL', '2026-01-16', 150.0, 'call',
    '2025-12-01', '14:30'
)
print(f"Greeks at 2:30 PM: \n{greeks}")

# With custom IV
greeks = greeks_intraday(
    'AAPL', '2026-01-16', 150.0, 'call',
    '2025-12-01', '14:30', iv=0.28
)
print(f"Greeks at 2:30 PM with IV = 28%: \n{greeks}")

In [ ]:
from functions import *

# Simulate buying a call option with 20% stop-loss and 50% take-profit
result = options_purchase(
    'AAPL', 
    '2026-01-16',  # Expiration
    150.0,          # Strike
    '2025-12-10',   # Purchase date
    '10:30',        # Purchase time (10:30 AM ET)
    'call',
    stoploss=20,    # -20% stop-loss
    takeprofit=50   # +50% take-profit
)

print(f"Entry: ${result['entry_price']}")
print(f"Exit: ${result['exit_price']} at {result['exit_time']}")
print(f"Reason: {result['exit_reason']}")
print(f"P/L: {result['pnl_percent']}% (${result['pnl_dollar']} per contract)")


In [ ]:
from functions import *

# Simulate buying a call option with 20% stop-loss and 50% take-profit
result = options_purchase(
    'AMC', 
    '2026-01-16',  # Expiration
    2,          # Strike
    '2025-12-16',   # Purchase date
    '09:30',        # Purchase time (09:30 AM ET)
    'call',
    stoploss=20,    # -20% stop-loss
    takeprofit=50   # +50% take-profit
)

print(f"Entry: ${result['entry_price']}")
print(f"Exit: ${result['exit_price']} at {result['exit_time']}")
print(f"Reason: {result['exit_reason']}")
print(f"P/L: {result['pnl_percent']}% (${result['pnl_dollar']} per contract)")


In [1]:
from functions import *

df = pd.read_csv('nasdaq_tickers.csv')

reasonable_stocks = []

for n in range(0,600):
    ticker, price = list(df[['Ticker','Last Price']].iloc[n])
    if 3*price < 120:
        reasonable_stocks.append([ticker, price])

reasonable_stocks = [[ticker, float(price)] for ticker, price in reasonable_stocks]

len(reasonable_stocks)

225

In [2]:
reasonable_stocks[0]

def check_option(ticker, price, date = '2026-01-16', margin = 0.01):
    try:
        # Get the option chain to see available strikes
        ticker_obj = yf.Ticker(ticker)
        chain = ticker_obj.option_chain(date)
        
        # Check if option chain has data
        if chain.calls.empty or chain.puts.empty:
            print(f"{ticker} - No option data available")
            return None
        
        # Find nearest strike for call (above current price)
        call_strikes = chain.calls['strike'].values
        call_target = price * (1 + margin)
        call_strike = call_strikes[np.argmin(np.abs(call_strikes - call_target))]
        #print(f"Call target: {call_target}, selected strike: {call_strike}")    

        # Find nearest strike for put (below current price)
        put_strikes = chain.puts['strike'].values
        put_target = price * (1 - margin)
        # Filter strikes below current price to ensure put is below stock price
        put_strikes_below = put_strikes[put_strikes < price]
        if len(put_strikes_below) > 0:
            put_strike = put_strikes_below[np.argmin(np.abs(put_strikes_below - put_target))]
        else:
            put_strike = put_strikes[np.argmin(np.abs(put_strikes - put_target))]
        #print(f"Put target: {put_target}, selected strike: {put_strike}")
        
        call_value = option_price(ticker, date, call_strike, 'call')
        call_value['strike'] = float(call_strike)
        call_value['ticker'] = ticker
        call_value['date'] = date

        put_value = option_price(ticker, date, put_strike, 'put')
        put_value['strike'] = float(put_strike)
        put_value['ticker'] = ticker
        put_value['date'] = date

        #print(f"{ticker} (${price:.2f})\n  Call ${call_strike}: {call_value}\n  Put ${put_strike}: {put_value}")
        return {'call': call_value  , 'put': put_value}
    
    except Exception as e:
        print(f"{ticker} - Error: {type(e).__name__}: {e}")
        return None

In [3]:
check_option("CMCSA", 30)['put']

{'lastPrice': 0.24,
 'bid': 0.2,
 'ask': 0.25,
 'volume': 3064,
 'strike': 27.5,
 'ticker': 'CMCSA',
 'date': '2026-01-16'}

In [4]:
greeks("CMCSA", '2026-01-16', 30, "put", status=True, silent = True)

{'delta': -0.42515902011427975,
 'gamma': 0.13590597006303737,
 'vega': 0.03349335689318717,
 'theta': -0.018140760267151546,
 'rho': -0.010997997330543444,
 'buy_score': 72,
 'recommendation': 'BUY'}

In [5]:
def stockings(stock_list, volume_threshold = None, buy_score_threshold = None):
    for ticker, price in stock_list: #197 entries
        info = check_option(ticker, price)
        if info is None:
            continue

        if volume_threshold is not None:
            if info['put']['volume'] < volume_threshold and info['call']['volume'] < volume_threshold:
                # print(f"Skipping {ticker} due to low volume (PUT: {info['put']['volume']}, CALL: {info['call']['volume']})")
                # print("-------------------------------------------------------")
                # print('')
                continue
        
        if buy_score_threshold is not None:
            put_buy_score = greeks(ticker, '2026-01-16', info['put']['strike'], "put", status=True, silent = True)['buy_score']
            call_buy_score = greeks(ticker, '2026-01-16', info['call']['strike'], "call", status=True, silent = True)['buy_score']
            if put_buy_score < buy_score_threshold and call_buy_score < buy_score_threshold:
                # print(f"Skipping {ticker} due to low buy score (PUT: {put_buy_score}%, CALL: {call_buy_score}%)")
                # print("-------------------------------------------------------")
                # print('')
                continue

        print(f"Analyzing {ticker} at price ${price}")

        print(f'Option contract price is approximately ${info["call"]["lastPrice"]*100}')
        print("")

        put_strike = info['put']['strike']
        call_strike = info['call']['strike']
        put_buy_score = greeks(ticker, '2026-01-16', info['put']['strike'], "put", status=True, silent = True)['buy_score']
        call_buy_score = greeks(ticker, '2026-01-16', info['call']['strike'], "call", status=True, silent = True)['buy_score']
        
        print(f'Buy score for PUT option at {put_strike}, with volume {info["put"]["volume"]} on 2026-01-16: {put_buy_score}%')
        print(f'Buy score for CALL option at {call_strike}, with volume {info["call"]["volume"]} on 2026-01-16: {call_buy_score}%')

        print('')
        print("-------------------------------------------------------")
        print('')


In [6]:
stockings(reasonable_stocks, volume_threshold=500, buy_score_threshold=70)

Analyzing CMCSA at price $29.73
Option contract price is approximately $125.0

Buy score for PUT option at 27.5, with volume 3064 on 2026-01-16: 44%
Buy score for CALL option at 30.0, with volume 11315 on 2026-01-16: 77%

-------------------------------------------------------

Analyzing NU at price $16.2
Option contract price is approximately $55.00000000000001

Buy score for PUT option at 16.0, with volume 388 on 2026-01-16: 85%
Buy score for CALL option at 16.0, with volume 898 on 2026-01-16: 77%

-------------------------------------------------------



KeyboardInterrupt: 

In [14]:
reasonable_stocks[0:1]

[['INTC', 37.31]]

In [2]:
pip install plotly panads

  Using cached plotly-6.5.0-py3-none-any.whl (9.9 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement panads (from versions: none)
ERROR: No matching distribution found for panads

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from fourier import *

In [ ]:
final_value = 10000.0

for h in range(1,30):
    for j in range(0,5):
        for k in range(2,10):
            for l in range(2,10):
                backtest_results = run_fourier_backtest(
                    ticker='SPY',
                    start_date='2025-01-01',
                    end_date='2025-12-20',
                    n_harmonics=h,
                    smoothing_sigma=j,
                    overbought_threshold=k,
                    oversold_threshold=-l,
                    initial_capital=10000.0
                )
                new_value = float(backtest_results['backtest']['final_value'])

                if new_value > final_value:
                    final_value = new_value
                    print(f"New best final value: ${final_value:.2f} with parameters:")
                    print(f"  n_harmonics={i}, smoothing_sigma={j}, overbought_threshold={k}, oversold_threshold={-l}")
                    print('')


[CACHE MISS] Downloading SPY data from 2025-01-01 to 2025-12-20...
[CACHE SAVED] Cached SPY data for future use
New best final value: $11161.10 with parameters:
  n_harmonics=1, smoothing_sigma=0, overbought_threshold=2, oversold_threshold=-2

New best final value: $11177.08 with parameters:
  n_harmonics=1, smoothing_sigma=0, overbought_threshold=2, oversold_threshold=-3

New best final value: $11687.32 with parameters:
  n_harmonics=1, smoothing_sigma=0, overbought_threshold=2, oversold_threshold=-4

New best final value: $12674.82 with parameters:
  n_harmonics=1, smoothing_sigma=0, overbought_threshold=2, oversold_threshold=-8

New best final value: $12744.55 with parameters:
  n_harmonics=1, smoothing_sigma=0, overbought_threshold=4, oversold_threshold=-8

New best final value: $12907.57 with parameters:
  n_harmonics=1, smoothing_sigma=0, overbought_threshold=5, oversold_threshold=-8

New best final value: $12999.72 with parameters:
  n_harmonics=1, smoothing_sigma=0, overbought_

In [ ]:
float(backtest_results['backtest']['final_value'])

12296.099838256836

In [24]:
backtest_results = run_fourier_backtest(
    ticker='NVDA',
    start_date='2025-10-01',
    end_date='2025-11-01',
    n_harmonics=11,
    smoothing_sigma=0,
    overbought_threshold=9,
    oversold_threshold=-8,
    initial_capital=10000.0
)

print_backtest_summary(backtest_results)

[CACHE MISS] Downloading NVDA data from 2025-10-01 to 2025-11-01...
[CACHE SAVED] Cached NVDA data for future use

BACKTEST SUMMARY: NVDA
Date Range: 2025-10-01 to 2025-11-01

Parameters:
  Harmonics: 11
  Smoothing σ: 0
  Overbought: 9
  Oversold: -8

Results:
  Initial Capital: $10,000.00
  Final Value: $11,496.55
  Total Return: 14.97%
  Number of Trades: 2
  Final Shares Held: 0
  Final Cash: $11,496.55

Recent Trades:
  2025-10-15 | BUY  |   55 shares @ $ 179.83 | oversold
  2025-10-29 | SELL |   55 shares @ $ 207.04 | overbought


In [21]:
analysis = backtest_results['fourier_analysis']
peaks, troughs = find_fourier_peaks(analysis, prominence=1.0)
print(f"\nFound {len(peaks)} peaks and {len(troughs)} troughs")


Found 6 peaks and 6 troughs


In [22]:
peaks

[FourierPeak(date=np.datetime64('2025-10-06T00:00:00.000000000'), index=np.int64(3), value=np.float64(7.262031141840399), is_peak=True, price_at_point=np.float64(607.7100219726562)),
 FourierPeak(date=np.datetime64('2025-10-08T00:00:00.000000000'), index=np.int64(5), value=np.float64(8.651376703970785), is_peak=True, price_at_point=np.float64(611.4400024414062)),
 FourierPeak(date=np.datetime64('2025-10-13T00:00:00.000000000'), index=np.int64(8), value=np.float64(-5.670603317278613), is_peak=True, price_at_point=np.float64(602.010009765625)),
 FourierPeak(date=np.datetime64('2025-10-15T00:00:00.000000000'), index=np.int64(10), value=np.float64(-8.167939746317014), is_peak=True, price_at_point=np.float64(602.219970703125)),
 FourierPeak(date=np.datetime64('2025-10-20T00:00:00.000000000'), index=np.int64(13), value=np.float64(-1.0212764943958774), is_peak=True, price_at_point=np.float64(611.5399780273438)),
 FourierPeak(date=np.datetime64('2025-10-29T00:00:00.000000000'), index=np.int64(

In [ ]:
# New best final value: $14603.96 with parameters:
#   n_harmonics=18, smoothing_sigma=0, overbought_threshold=9, oversold_threshold=-8


In [ ]:
    # ticker : str
    #     Stock ticker symbol
    # start_date, end_date : str
    #     Date range in 'YYYY-MM-DD' format
    # n_harmonics : int
    #     Number of Fourier harmonics (default: 10)
    # smoothing_sigma : float
    #     Smoothing parameter (default: 2.0)
    # overbought_threshold, oversold_threshold : float
    #     Trading thresholds for signal detection
    # initial_capital : float
    #     Starting capital (default: $10,000)
    # contracts_per_trade : int
    #     Number of option contracts per trade (default: 1)
    # stoploss_percent : float
    #     Stop-loss percentage (default: 50%)
    # takeprofit_percent : float
    #     Take-profit percentage (default: 50%)
    # days_to_expiry : int
    #     Days to option expiration (default: 30)
    # otm_percent : float
    #     Out-of-the-money percentage (default: 2%)
    # max_positions : int
    #     Maximum open positions (default: 1)
    # tick_size : str
    #     Data interval (default: '1d')
    # verbose : bool
    #     Print detailed information (default: True)

In [2]:
from yfinance_cache import download_cached

# Pre-cache SPY data for your entire date range
data = download_cached('SPY', '2025-01-01', '2025-05-01', interval='1d')
print(f"Cached {len(data)} days of SPY data")


Cached 81 days of SPY data


In [13]:
from fourier import *

results = run_fourier_options_backtest(
    ticker='SPY',
    start_date='2025-11-01',
    end_date='2025-12-01',
    n_harmonics=30,
    smoothing_sigma=0,
    overbought_threshold=9,
    oversold_threshold=-8,
    contracts_per_trade=1,
    max_positions=2,
    tick_size='1h',
    stoploss_percent=50,
    takeprofit_percent=50,
    days_to_expiry=30,
    otm_percent=2.0,
    initial_capital=10000
)

# print(f"Total Return: {results.total_return:.2f}%")
# print(f"Win Rate: {results.win_rate:.1f}%")


Detected 9 signals
Buy signals: 5
Sell signals: 4

OPTIONS BACKTESTING: SPY
Initial Capital: $10,000.00
Contracts per Trade: 1
Stop-Loss: 50% | Take-Profit: 50%
Days to Expiry: 30 | OTM: 2.0%
Max Open Positions: 2



Processing signals:  33%|███▎      | 3/9 [00:00<00:00, 299.87signal/s, Pricing put $669]  


[2025-11-07] OPENED: CALL $679 exp 2025-12-07
  Stock: $665.93 | Entry: $7.73 | Cost: $773.38
  Signal: oversold | Capital: $9,226.62
  Greeks - Delta: 0.418, Gamma: 0.0150, Theta: -0.198, Vega: 0.751
[2025-11-11] TAKE-PROFIT: CALL $679 exp 2025-12-07
  Entry: $7.73 → Exit: $13.67 (76.7%)

[2025-11-11] OPENED: PUT $669 exp 2025-12-11
  Stock: $683.13 | Entry: $4.94 | Cost: $494.18
  Signal: overbought | Capital: $10,099.23
  Greeks - Delta: -0.276, Gamma: 0.0118, Theta: -0.138, Vega: 0.655
[CACHE MISS] Downloading SPY data from 2025-11-07 to 2025-11-13...
[CACHE SAVED] Cached SPY data for future use


c:\Users\tawfe\Documents\GitHub\trading\functions.py:694: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead

c:\Users\tawfe\Documents\GitHub\trading\technical_retrievals.py:128: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead

Processing signals:  56%|█████▌    | 5/9 [00:00<00:00,  9.01signal/s, Pricing put $670] 

[CACHE MISS] Downloading SPY data from 2025-10-03 to 2025-11-12...
[CACHE SAVED] Cached SPY data for future use

[2025-11-12] OPENED: PUT $670 exp 2025-12-12
  Stock: $683.20 | Entry: $5.13 | Cost: $513.39
  Signal: overbought | Capital: $9,585.85
  Greeks - Delta: -0.284, Gamma: 0.0119, Theta: -0.140, Vega: 0.664
[2025-11-17] TAKE-PROFIT: PUT $669 exp 2025-12-11
  Entry: $4.94 → Exit: $11.27 (128.0%)


Processing signals:  89%|████████▉ | 8/9 [00:00<00:00,  9.01signal/s, Pricing call $672]c:\Users\tawfe\Documents\GitHub\trading\functions.py:694: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



[2025-11-17] TAKE-PROFIT: PUT $670 exp 2025-12-12
  Entry: $5.13 → Exit: $11.98 (133.4%)

[2025-11-17] OPENED: CALL $679 exp 2025-12-17
  Stock: $665.69 | Entry: $6.95 | Cost: $695.10
  Signal: oversold | Capital: $11,215.42
  Greeks - Delta: 0.359, Gamma: 0.0128, Theta: -0.206, Vega: 0.713

[2025-11-19] OPENED: CALL $674 exp 2025-12-19
  Stock: $661.17 | Entry: $6.11 | Cost: $611.01
  Signal: oversold | Capital: $10,604.41
  Greeks - Delta: 0.364, Gamma: 0.0150, Theta: -0.181, Vega: 0.713
[2025-11-20] STOP-LOSS: CALL $679 exp 2025-12-17
  Entry: $6.95 → Exit: $1.55 (-77.7%)
[2025-11-20] STOP-LOSS: CALL $674 exp 2025-12-19
  Entry: $6.11 → Exit: $2.59 (-57.6%)

[2025-11-20] OPENED: CALL $672 exp 2025-12-20
  Stock: $658.57 | Entry: $3.14 | Cost: $313.90
  Signal: oversold | Capital: $10,704.62
  Greeks - Delta: 0.235, Gamma: 0.0133, Theta: -0.134, Vega: 0.575
[CACHE MISS] Downloading SPY data from 2025-11-16 to 2025-11-22...
[CACHE SAVED] Cached SPY data for future use
[CACHE MISS] Dow

c:\Users\tawfe\Documents\GitHub\trading\technical_retrievals.py:128: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead

Processing signals: 100%|██████████| 9/9 [00:01<00:00,  8.72signal/s, Opening put $665] 

1 Failed download:
['SPY']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-12-21 -> 2025-12-27)')


[CACHE SAVED] Cached SPY data for future use
[2025-11-21] TAKE-PROFIT: CALL $672 exp 2025-12-20
  Entry: $3.14 → Exit: $5.20 (65.8%)

[2025-11-21] OPENED: CALL $678 exp 2025-12-21
  Stock: $664.23 | Entry: $3.69 | Cost: $368.90
  Signal: oversold | Capital: $10,856.09
  Greeks - Delta: 0.255, Gamma: 0.0131, Theta: -0.148, Vega: 0.606
[2025-11-26] TAKE-PROFIT: CALL $678 exp 2025-12-21
  Entry: $3.69 → Exit: $11.93 (223.4%)

[2025-11-26] OPENED: PUT $665 exp 2025-12-26
  Stock: $678.13 | Entry: $4.59 | Cost: $458.96
  Signal: overbought | Capital: $11,590.31
  Greeks - Delta: -0.265, Gamma: 0.0117, Theta: -0.133, Vega: 0.638

Backtest Cache Statistics:
  Option price cache: 17 unique calculations
  Greeks cache: 15 unique calculations

API Call Cache Statistics:
  Stock prices: 130 hits, 36 misses (78.3% hit rate)
  Interest rates: 83 hits, 30 misses (73.5% hit rate)
  Volatility: 97 hits, 16 misses (85.8% hit rate)
[CACHE MISS] Downloading SPY data from 2025-12-21 to 2025-12-27...

BACK

In [14]:
print_options_backtest_summary(results)

BACKTEST SUMMARY: SPY
Date Range: 2025-11-03 to 2025-11-28

Capital:
  Initial: $10,000.00
  Final: $11,590.31
  Total Return: 15.90%

Trades:
  Total: 8
  Winners: 5 (62.5%)
  Losers: 2
  Avg P&L: 70.28%
  Avg Days Held: 3.3

Parameters:
  contracts_per_trade: 1
  stoploss_percent: 50
  takeprofit_percent: 50
  days_to_expiry: 30
  otm_percent: 2.0
  max_positions: 2

Sample Trades (last 5):
Date         Type   Strike   Entry    Exit     P&L %      Days   Reason      
--------------------------------------------------------------------------------
2025-11-17   CALL   $679     $6.95    $1.55    -77.7    % 2      stoploss    
2025-11-19   CALL   $674     $6.11    $2.59    -57.6    % 1      stoploss    
2025-11-20   CALL   $672     $3.14    $5.20    65.8     % 1      takeprofit  
2025-11-21   CALL   $678     $3.69    $11.93   223.4    % 4      takeprofit  
2025-11-26   PUT    $665     $4.59    $0.00    0.0      % 0      open        

